In [ ]:
import pandas as pd
import scheme_mar2023 as scheme

### recovery algorithm

Computes real gas concentration in water  
 
> input parameters:  
> - **datetime** : *col in pd.DataFrame timestamp* - date time
> - **v_gas_equ** : *col in pd.DataFrame* - gas mixing ratio in equilibrator (ppmv)
> - **v_gas_air** : *col in pd.DataFrame* - gas mixing ratio in air. inflow gas into equilibrator while sampling (ppmv)
> - **t_wrt** : *col in pd.DataFrame* - water temperature in equilibrator. Air temperature is equal (grad K)
> - **press** : *col in pd.DataFrame* - air pressure (atm)
> - **v_air** : *col in pd.DataFrame* - air flow thrw equilibrator (l/min)
> - **v_wtr** : *col in pd.DataFrame* - water flow thrw equilibrator (l/min)
> - **gas_type** : *String* - gas formulae: 'CO2', 'CH4'
> - **equ_param** : *dict* - equilibratore features, as `equ_vol`, `equ_eff`
>
> output:  
> - pd.series gas in water concentration (mg/l)
 

In [ ]:
    """ add columns to df with names 'c'+DefName and 'p'+DefName filled with
    recovered values in [g/l] and [uatm] accordingly """

def equToWtrRecovery(datetime, v_gas_equ, v_gas_air, t_wtr, press, v_air, v_wtr, equ_param, gas_type='CO2'):
    dt = datetime.shift(1) - datetime  # time delta to int of minutes
    equ_vol = equ_param['equ_vol']  # equivalent equ volume, l
    equ_eff = equ_param['equ_eff']  # equilibrator capacity
    print('Equ volume = {}, equ capasity = {}'.format(equ_vol, equ_eff))

    sol = getSolubility(t_wtr, gas_type)
    t_air = t_wtr
    # !!!
    den = getDensity(t_air, None, gas_type)
    press = pressConvert(press)

    # !!!
    cGasAir = vGasAir.median()*0.000001*density*press/NORM_PRESS  # g/l

    pGasEqu = vGasEqu*press/NORM_PRESS
    cGasEqu = pGasEqu*density*0.000001  # g/l

    tau = equ_vol/(air_flow+wtrflow*equ_eff*solubility/density)
    eternal = (cGasEqu-cGasEqu.shift(1)*np.exp(-1*dt/tau))/(1-np.exp(-1*dt/tau))
    cGasWtr = (eternal*(wtrflow*equ_eff*solubility/density+air_flow)-air_flow*cGasAir)/(wtrflow*equ_eff)
    temp_col_name = 'c'+out_col_name
    df[temp_col_name] = cGasWtr
    print(temp_col_name)
    temp_col_name = 'p'+out_col_name
    df[temp_col_name] = cGasWtr*1000000/solubility
    print(temp_col_name)
